In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tomoro-ai/combined.csv
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer_config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model.bin.index.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/special_tokens_map.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/.gitattributes
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.model
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/generation_config.json


In [3]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets trl
!pip install -q -U huggingface_hub einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 22.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 103.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 8.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 3.9 MB/s eta 0:00:00


In [4]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig
)


In [ ]:
# !python -m pip install -U peft

In [5]:
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer
import torch

RuntimeError: Failed to import trl.trainer.sft_trainer because of the following error (look up to see its traceback):
Failed to import transformers.processing_utils because of the following error (look up to see its traceback):
cannot import name 'load_video' from 'transformers.image_utils' (/usr/local/lib/python3.10/dist-packages/transformers/image_utils.py)

In [6]:
model_name="/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

In [8]:
# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)


In [9]:
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from huggingface_hub import login
login("hf_GKpiaEsGfLnGpTrODlWTeoeznSZabBOF")

In [18]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer.pad_token = tokenizer.eos_token
prompt="""Question: what was the percentage change in the net cash from operating activities from 2008 to 2009
Pre-text: 26 | 2009 annual report in fiscal 2008, revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007. all revenue components within the segment experienced growth during fiscal 2008. license revenue generated the largest dollar growth in revenue as episys ae, our flagship core processing system aimed at larger credit unions, experienced strong sales throughout the year. support and service revenue, which is the largest component of total revenues for the credit union segment, experienced 34 percent growth in eft support and 10 percent growth in in-house support. gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007, due primarily to the increase in license revenue, which carries the highest margins. liquidity and capital resources we have historically generated positive cash flow from operations and have generally used funds generated from operations and short-term borrowings on our revolving credit facility to meet capital requirements. we expect this trend to continue in the future. the company 2019s cash and cash equivalents increased to $ 118251 at june 30, 2009 from $ 65565 at june 30, 2008. the following table summarizes net cash from operating activities in the statement of cash flows : 2009 2008 2007.
Table:
<table class='wikitable'><tr><td>1</td><td>2008</td><td>year ended june 30 2009 2008</td><td>year ended june 30 2009 2008</td><td>year ended june 30 2009</td></tr><tr><td>2</td><td>net income</td><td>$ 103102</td><td>$ 104222</td><td>$ 104681</td></tr><tr><td>3</td><td>non-cash expenses</td><td>74397</td><td>70420</td><td>56348</td></tr><tr><td>4</td><td>change in receivables</td><td>21214</td><td>-2913 ( 2913 )</td><td>-28853 ( 28853 )</td></tr><tr><td>5</td><td>change in deferred revenue</td><td>21943</td><td>5100</td><td>24576</td></tr><tr><td>6</td><td>change in other assets and liabilities</td><td>-14068 ( 14068 )</td><td>4172</td><td>17495</td></tr><tr><td>7</td><td>net cash from operating activities</td><td>$ 206588</td><td>$ 181001</td><td>$ 174247</td></tr></table>
Post-text: year ended june 30, cash provided by operations increased $ 25587 to $ 206588 for the fiscal year ended june 30, 2009 as compared to $ 181001 for the fiscal year ended june 30, 2008. this increase is primarily attributable to a decrease in receivables compared to the same period a year ago of $ 21214. this decrease is largely the result of fiscal 2010 annual software maintenance billings being provided to customers earlier than in the prior year, which allowed more cash to be collected before the end of the fiscal year than in previous years. further, we collected more cash overall related to revenues that will be recognized in subsequent periods in the current year than in fiscal 2008. cash used in investing activities for the fiscal year ended june 2009 was $ 59227 and includes $ 3027 in contingent consideration paid on prior years 2019 acquisitions. cash used in investing activities for the fiscal year ended june 2008 was $ 102148 and includes payments for acquisitions of $ 48109, plus $ 1215 in contingent consideration paid on prior years 2019 acquisitions. capital expenditures for fiscal 2009 were $ 31562 compared to $ 31105 for fiscal 2008. cash used for software development in fiscal 2009 was $ 24684 compared to $ 23736 during the prior year. net cash used in financing activities for the current fiscal year was $ 94675 and includes the repurchase of 3106 shares of our common stock for $ 58405, the payment of dividends of $ 26903 and $ 13489 net repayment on our revolving credit facilities. cash used in financing activities was partially offset by proceeds of $ 3773 from the exercise of stock options and the sale of common stock ( through the employee stock purchase plan ) and $ 348 excess tax benefits from stock option exercises. during fiscal 2008, net cash used in financing activities for the fiscal year was $ 101905 and includes the repurchase of 4200 shares of our common stock for $ 100996, the payment of dividends of $ 24683 and $ 429 net repayment on our revolving credit facilities. cash used in financing activities was partially offset by proceeds of $ 20394 from the exercise of stock options and the sale of common stock and $ 3809 excess tax benefits from stock option exercises. beginning during fiscal 2008, us financial markets and many of the largest us financial institutions have been shaken by negative developments in the home mortgage industry and the mortgage markets, and particularly the markets for subprime mortgage-backed securities. since that time, these and other such developments have resulted in a broad, global economic downturn. while we, as is the case with most companies, have experienced the effects of this downturn, we have not experienced any significant issues with our current collection efforts, and we believe that any future impact to our liquidity will be minimized by cash generated by recurring sources of revenue and due to our access to available lines of credit. 
 Generate Answer of the given question using Pre-text, Table and Post-text. You break the question into simpler part and generate it's response"""
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
input_ids = inputs.input_ids
attention_mask = inputs.attention_mask
with torch.no_grad():  # Disable gradient calculations

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,  # Include attention mask here
        max_new_tokens=270,
            use_cache=False  # Disable caching if it's causing issues

    )


# Decode and format output
full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Full Response")
print(full_response)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Full Response
Question: what was the percentage change in the net cash from operating activities from 2008 to 2009
Pre-text: 26 | 2009 annual report in fiscal 2008, revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007. all revenue components within the segment experienced growth during fiscal 2008. license revenue generated the largest dollar growth in revenue as episys ae, our flagship core processing system aimed at larger credit unions, experienced strong sales throughout the year. support and service revenue, which is the largest component of total revenues for the credit union segment, experienced 34 percent growth in eft support and 10 percent growth in in-house support. gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007, due primarily to the increase in license revenue, which carries the highest margins. liquidity and capital resources we have historically generated positive cash flo

In [19]:
import pandas as pd

In [20]:
df=pd.read_csv("/kaggle/input/tomoro-ai/combined.csv")

In [21]:
df["Processed"][0]

"Question: what was the percentage change in the net cash from operating activities from 2008 to 2009\nPre-text: 26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 . all revenue components within the segment experienced growth during fiscal 2008 . license revenue generated the largest dollar growth in revenue as episys ae , our flagship core processing system aimed at larger credit unions , experienced strong sales throughout the year . support and service revenue , which is the largest component of total revenues for the credit union segment , experienced 34 percent growth in eft support and 10 percent growth in in-house support . gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007 , due primarily to the increase in license revenue , which carries the highest margins . liquidity and capital resources we have historically generated positive cash flo

In [22]:
# Prepare model for training
model = prepare_model_for_kbit_training(model)

In [23]:
peft_config = LoraConfig(
    r=128,
    lora_alpha=128,
    lora_dropout=0.01,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]  # Apply LoRA to more layers
)


In [27]:
#%% [Training Arguments]
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=30,
    per_device_train_batch_size=8,  # Reduce if OOM occurs
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    fp16=True,
    logging_steps=10,
    optim="paged_adamw_8bit",
    save_strategy="steps",
    save_steps=250,
    save_total_limit=1,  # Keep only the latest model, replacing the previous one
    report_to="none",
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    dataloader_pin_memory=True,  # GPU memory optimization

)


In [43]:
prompt=". Given the following context (pretext, posttext, and table), calculate the solution of given question with the help of the context. Break the solution into multiple dialogue turns, showing intermediate calculations step by step, and provide a final program representation of the calculations. "
df['new_process'] = df['Processed'].apply(lambda text: text.split("Dialogue")[0] + prompt+ text.replace(text.split("Dialogue")[0], "", 1))

In [47]:
df["new_process"][3]

"Question: what was the percent of the change in the company 2019s warranty liability from 2011 to 2012\nPre-text: masco corporation notes to consolidated financial statements ( continued ) t . other commitments and contingencies litigation . we are subject to claims , charges , litigation and other proceedings in the ordinary course of our business , including those arising from or related to contractual matters , intellectual property , personal injury , environmental matters , product liability , construction defect , insurance coverage , personnel and employment disputes and other matters , including class actions . we believe we have adequate defenses in these matters and that the outcome of these matters is not likely to have a material adverse effect on us . however , there is no assurance that we will prevail in these matters , and we could in the future incur judgments , enter into settlements of claims or revise our expectations regarding the outcome of these matters , which 

In [41]:
df["Processed"][2]

"Question: what portion of the total shares subject to outstanding awards is under the 2009 global incentive plan?\nPre-text: tax returns for 2001 and beyond are open for examination under statute . currently , unrecognized tax benefits are not expected to change significantly over the next 12 months . 19 . stock-based and other management compensation plans in april 2009 , the company approved a global incentive plan which replaces the company 2019s 2004 stock incentive plan . the 2009 global incentive plan ( 201cgip 201d ) enables the compensation committee of the board of directors to award incentive and nonqualified stock options , stock appreciation rights , shares of series a common stock , restricted stock , restricted stock units ( 201crsus 201d ) and incentive bonuses ( which may be paid in cash or stock or a combination thereof ) , any of which may be performance-based , with vesting and other award provisions that provide effective incentive to company employees ( including 

In [ ]:
dataset = Dataset.from_pandas(df)


In [ ]:
#%% [Initialize Trainer]
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    # dataset_text_field='Processed',
    # max_seq_length=2048,  # Reduce based on your context length
    # tokenizer=tokenizer,
    formatting_func=lambda example: example["new_process"],
    args=training_args,
    # packing=False
)


In [29]:
#%% [Start Training]
trainer.train()

#%% [Save Adapter]
trainer.save_model("final_model")


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


KeyboardInterrupt: 

In [ ]:
prompt=""
df['Processed'] = df['Processed'] + prompt
